In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter

from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Train set load & split

In [2]:
df = pd.read_csv('train.csv')

# dataframe을 series로 나누어 label과 data로 관리
X_data = df['comment']
y_data = df['toxicity']

# stratify -> 레이블 비율 일정히 trainset과 validset에 분배
# 검증을 위한 validset을 준비
X_train, X_valid, y_train, y_valid = train_test_split(X_data, y_data, test_size=.2, random_state=0, stratify=y_data)


# Tokenize

In [3]:
# nttk의 tokenize함수를 통해 white tokenize 수행 후 
# comment 별로 소문자화 후에 이중list에 저장
def tokenize(sentences):
  tokenized_sentences = []
  for sent in tqdm(sentences):
    tokenized_sent = word_tokenize(sent)
    tokenized_sent = [word.lower() for word in tokenized_sent]
    tokenized_sentences.append(tokenized_sent)
  return tokenized_sentences

tokenized_X_train = tokenize(X_train)
tokenized_X_valid = tokenize(X_valid)



100%|██████████████████████████████████████████████████████████████████████████| 15957/15957 [00:07<00:00, 2125.39it/s]


# Vocab

In [4]:
# vocab 구성을 위해 voc_list를 구성 후 counter 객체로 갯수를 맵핑
# 중간 과제 vocab 생성 코드 참조하였음 (NGram.py)
voc_list = []
for sent in tokenized_X_train:
    for word in sent:
      voc_list.append(word)
counters = Counter(voc_list)
print('총 단어수 :', len(counters))
# 이후 sorted 함수를 이용해 빈도수가 높은 단어순으로 정렬
vocab = sorted(counters, key=counters.get, reverse=True)



min_occur = 3
total_cnt = len(counters) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트

for _, value in counters.items():
    # 단어빈도가 min_occur보다 작으면
    if(value < min_occur):
        rare_cnt = rare_cnt + 1
        

print('min_occur 제거 전 단어집합 크기 :',total_cnt)
# 크기로 정렬된 vocab에 index sliciing으로 min_occur voc 제거
vocab_size = total_cnt - rare_cnt
vocab = vocab[:vocab_size]
print('min_occur 제거 후 단어집합 크기 :', len(vocab))




# 단어집합 embedding 구성을 위해 빈도를 활용 ( encoding 된 단어는 emdedding층을 만나고 훈련과정에서 vector로 맵핑)
# 단순 dict로 빈도수를 맵핑
# padding과 unknown words vocab index 구성, 이를 위해 idx값을 뒤로 두칸씩
word_to_index = {}
word_to_index['<PAD>'] = 0
word_to_index['<UNK>'] = 1

for index, word in enumerate(vocab) :
  word_to_index[word] = index + 2

vocab_size = len(word_to_index)
print('최종 단어 집합의 크기 :', vocab_size)




총 단어수 : 140077
min_occur 제거 전 단어집합 크기 : 140077
min_occur 제거 후 단어집합 크기 : 38867
최종 단어 집합의 크기 : 38869


# encoding

In [5]:
def texts_to_sequences(tokenized_X_data, word_to_index):
  encoded_X_data = []
  for sent in tokenized_X_data:
    index_sequences = []
    for word in sent:
      try:
          index_sequences.append(word_to_index[word])
      except KeyError:
          index_sequences.append(word_to_index['<UNK>'])
    encoded_X_data.append(index_sequences)
  return encoded_X_data
# vocab table을 활용하여 dataset의 단어를 정수로 mapping
# 해당 함수는 로직은 쉬우나 exception 처리로 구성된 코드를 참조 링크에서 참조하였음
encoded_X_train = texts_to_sequences(tokenized_X_train, word_to_index)
encoded_X_valid = texts_to_sequences(tokenized_X_valid, word_to_index)

for sent in encoded_X_train[:2]:
  print(sent)
# word_to_index의 key,value pair를 value,key pari로 저장하여 decoding에 활용할 dict 구성
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key

decoded_sample = [index_to_word[word] for word in encoded_X_train[0]]
print('기존의 첫번째 샘플 :', tokenized_X_train[0])
print('복원된 첫번째 샘플 :', decoded_sample)

[573, 63, 15, 30, 68, 46, 1, 162, 3, 1385, 2, 568, 7, 91, 1054, 184, 4, 15, 87, 37, 713, 13, 268, 30, 501, 15, 96]
[14, 71, 22, 422, 146, 5, 14, 6, 878, 1815, 643, 9, 271, 331, 1697, 143, 6, 6, 7, 59, 19, 965, 5, 3, 227, 452, 2413, 46, 14, 6, 3, 213, 75, 317, 341, 20, 2729, 96, 6, 6, 2, 26, 18, 3, 3469, 8, 3, 287, 24, 308, 2, 634, 8, 1968, 469, 5, 25622, 2, 30715, 2598, 5, 3, 36, 43, 412, 13, 62, 8, 3, 634, 8, 3, 1968, 469, 35, 3, 175, 25622, 30, 35, 71, 6866, 45, 1456, 4075, 108, 9953, 2, 20, 398, 54, 73, 1637, 16, 3, 786, 8, 3, 994, 4, 39, 7, 27, 19, 460, 20, 398, 146, 16, 69, 791, 134, 2, 34, 20, 12, 556, 5, 41, 2358, 4, 7, 481, 30715, 1731, 46, 318, 73, 292, 4, 38, 4, 34, 58, 248, 37, 157, 103, 233, 870, 4, 46, 318, 1022, 73, 8068, 138, 58, 706, 20, 474, 4, 49, 13, 15, 47, 25, 1475, 2, 78, 455, 22, 3, 53, 40, 13, 14, 6, 148, 2066, 6867, 6, 6, 22, 20, 12, 174, 3061, 8, 11, 291, 2, 34, 20, 12, 683, 78, 182, 2366, 437, 22, 78, 1794, 8, 3, 1637, 8, 994, 4, 20, 3527, 26, 294, 379, 4, 12

# padding for unequal size

In [6]:
# comment 별 size가 다르기에 데이터 차원을 통일 시켜야한다.

print('리뷰의 최대 길이 :',max(len(review) for review in encoded_X_train))
# data를 len() 함수로 맵핑하여 간단히 dataset의 길이의 합을 구하고 평균을 구하는 코드
print('리뷰의 평균 길이 :',sum(map(len, encoded_X_train))/len(encoded_X_train))

리뷰의 최대 길이 : 4948
리뷰의 평균 길이 : 80.12867393620354


In [7]:
# sample의 길이의 비율을 구하는 함수
# comment 최대길이가 5000인데 비해
# 평균길이는 80이어서 적절 패딩사이즈를 구하기 위함
# 직접 max_len값을 넣어주며 적절한 padding 양을 고려하였음

def len_ratio(max_len, data):
  cnt = 0
  for sent in data:
    if(len(sent) <= max_len):
        cnt = cnt + 1
  print(f"길이가 {max_len}이하인 샘플의 비율: {(cnt/len(data))*100}")

max_len = 700
len_ratio(max_len, encoded_X_train)
# 700으로 하여도 전체 99%가량의 data를 cover 하였음

길이가 700이하인 샘플의 비율: 99.09130788995425


In [8]:
# np array로 데이터 feature 생성 및 인자값에 맞게 padding
def pad_sequences(sentences, max_len):
  features = np.zeros((len(sentences), max_len), dtype=int)
  for index, sentence in enumerate(sentences):
    if len(sentence) != 0:
      features[index, :len(sentence)] = np.array(sentence)[:max_len]
        # sentence 길이에 해당하는 값을 feature array에 넣어준다, array 복사 연산 또한 차원을 맞춰주었음
  return features

padded_X_train = pad_sequences(encoded_X_train, max_len=max_len)
padded_X_valid = pad_sequences(encoded_X_valid, max_len=max_len)

print('훈련 데이터의 크기 :', padded_X_train.shape)
print('검증 데이터의 크기 :', padded_X_valid.shape)

훈련 데이터의 크기 : (63828, 700)
검증 데이터의 크기 : (15957, 700)


# Model

In [9]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print("사용 device:", device)

사용 device: cuda


In [10]:
# label data를 우선 np array로 만들고 tensor로 변환
train_label_tensor = torch.tensor(np.array(y_train))
valid_label_tensor = torch.tensor(np.array(y_valid))
print(train_label_tensor[:30])


# 최종 단어 집합크기, em_dim, hid_dim, out_dim (label 2개에 맞추어 2로 설정) , dropout_p -> 50% 제공
# Embedding 층을 통해 단어집합을 vector로구성
# lstm 모델을 사용, 0차원을 batch로구성
# 최종 output 출력을 위한 linear layer -> 2
class TextModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout_p = 0.5):
        super(TextModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout_p = dropout_p

    def forward(self, x):
        embedded = self.embedding(x)  # (batch_size, seqence 길이) -> (batch_size, seqence 길이, embedding_dim)

        # LSTM은 (hidden state, cell state)를 반환한다
        lstm_out, (hidden, cell) = self.lstm(embedded)  # lstm_out: (batch_size, seqence 길이, hidden_dim), hidden: (1, batch_size, hidden_dim)

        last_hidden = hidden.squeeze(0)  # (batch_size, hidden_dim)
        logits = self.fc(last_hidden)  # (batch_size, output_dim) # 최종 2개의 label값에 대한 추론
        return logits

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0])


# Train Setting

In [11]:
encoded_train = torch.tensor(padded_X_train).to(torch.int64)
train_dataset = torch.utils.data.TensorDataset(encoded_train, train_label_tensor)
train_dataloader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=32)
# train set 및 valid set또한 학습을 위해 tensor로 변환, 이후
# dataset module을 통해 학습 데이터 구성
# dataloader를 통해 batch_size 32으로 학습진행 구성
# valid set의 경우 batch_size 1로
encoded_valid = torch.tensor(padded_X_valid).to(torch.int64)
valid_dataset = torch.utils.data.TensorDataset(encoded_valid, valid_label_tensor)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, shuffle=True, batch_size=1)

total_batch = len(train_dataloader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 1995


In [12]:
# hyper param 설정
# epoch의 경우 실험을 통해 accuracy가 준수한 수치를 채택하였음
embedding_dim = 100
hidden_dim = 128
output_dim = 2
learning_rate = 0.01
num_epochs = 10

model = TextModel(vocab_size, embedding_dim, hidden_dim, output_dim)
model.to(device)
# loss function으로 crossentropy, optimizer는 Adam으로 구성
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Evaluation

In [13]:
def calculate_accuracy(logits, labels):
    # argmax -> dim =1 이면 행기준 max값의 index 출력
    # 해당값이 label과 일치할 경우의 합을 더하여 accuracy return
    predicted = torch.argmax(logits, dim=1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = correct / total
    return accuracy

In [14]:
def evaluate(model, valid_dataloader, criterion, device):
    val_loss = 0
    val_correct = 0
    val_total = 0

    model.eval()
    # 가중치 갱신 없이 (검증)
    with torch.no_grad():
        # valid set 에서  배치(1)만큼의 데이터 load
        for batch_X, batch_y in valid_dataloader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            # 모델의 예측값
            logits = model(batch_X)

            # 손실을 계산
            loss = criterion(logits, batch_y)

            # 정확도와 손실을 계산함
            val_loss += loss.item()
            val_correct += calculate_accuracy(logits, batch_y) * batch_y.size(0)
            val_total += batch_y.size(0)

    val_accuracy = val_correct / val_total
    val_loss /= len(valid_dataloader)
    # ross 와 accuracy return
    return val_loss, val_accuracy

# Train Session

In [15]:
num_epochs = 10

# 검증 loss init
best_val_loss = float('inf')

# Training loop
# tqdm으로 진행상태 표시
for epoch in tqdm(range(num_epochs)):
    # Training
    train_loss = 0
    train_correct = 0
    train_total = 0
    model.train()
    for batch_X, batch_y in train_dataloader:
        # Forward pass (32 size batch)
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        # batch_X.shape == (batch_size, max_len)
        logits = model(batch_X)

        # loss 계산
        loss = criterion(logits, batch_y)
        # 가중치 초기화
        optimizer.zero_grad()
        # 오류 역전파
        loss.backward()
        optimizer.step()

        # Calculate training accuracy and loss
        train_loss += loss.item()
        train_correct += calculate_accuracy(logits, batch_y) * batch_y.size(0)
        train_total += batch_y.size(0)

    train_accuracy = train_correct / train_total
    train_loss /= len(train_dataloader)

    # Validation
    val_loss, val_accuracy = evaluate(model, valid_dataloader, criterion, device)

    print(f'Epoch {epoch+1}/{num_epochs}:')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # 검증 손실이 최소일 때 체크포인트 저장
    if val_loss < best_val_loss:
        print(f'Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. 체크포인트를 저장합니다.')
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model_checkpoint.pth')

 10%|████████▎                                                                          | 1/10 [01:11<10:39, 71.07s/it]

Epoch 1/10:
Train Loss: 0.3168, Train Accuracy: 0.9044
Validation Loss: 0.3152, Validation Accuracy: 0.9046
Validation loss improved from inf to 0.3152. 체크포인트를 저장합니다.



 20%|████████████████▌                                                                  | 2/10 [02:19<09:13, 69.23s/it]

Epoch 2/10:
Train Loss: 0.3148, Train Accuracy: 0.9048
Validation Loss: 0.3149, Validation Accuracy: 0.9046
Validation loss improved from 0.3152 to 0.3149. 체크포인트를 저장합니다.



 30%|████████████████████████▉                                                          | 3/10 [03:28<08:06, 69.57s/it]

...

Epoch 8/10:
Train Loss: 0.1458, Train Accuracy: 0.9437
Validation Loss: 0.1419, Validation Accuracy: 0.9495
Validation loss improved from 0.1610 to 0.1419. 체크포인트를 저장합니다.



 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [10:39<01:11, 71.40s/it]

Epoch 9/10:
Train Loss: 0.1025, Train Accuracy: 0.9633
Validation Loss: 0.1300, Validation Accuracy: 0.9552
Validation loss improved from 0.1419 to 0.1300. 체크포인트를 저장합니다.


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [11:51<00:00, 71.15s/it]

Epoch 10/10:
Train Loss: 0.0789, Train Accuracy: 0.9719
Validation Loss: 0.1344, Validation Accuracy: 0.9573


In [16]:
# 모델 로드
model.load_state_dict(torch.load('best_model_checkpoint.pth'))

# 모델을 device에 올립니다.
model.to(device)
# 검증 데이터에 대한 정확도와 손실 계산
val_loss, val_accuracy = evaluate(model, valid_dataloader, criterion, device)

print(f'Best validation loss: {val_loss:.4f}')
print(f'Best validation accuracy: {val_accuracy:.4f}')


Best validation loss: 0.1300
Best validation accuracy: 0.9552


# Test Session

In [17]:
index_to_tag = {0 : 'normal', 1 : 'toxicity'}

def predict(text, model, word_to_index, index_to_tag):
    # 모델 평가 모드
    model.eval()

    # test sentence를 단순 white tokenize
    # unknown word는 1 할당
    tokens = word_tokenize(text)
    token_indices = [word_to_index.get(token.lower(), 1) for token in tokens]

    # 리스트를 텐서로 변환
    input_tensor = torch.tensor([token_indices], dtype=torch.long).to(device)  # (1, seq_length)

    # 당연히 가중치 갱신없이 예측
    with torch.no_grad():
        logits = model(input_tensor)  # (1, output_dim)
        
    probs = F.softmax(logits, dim=1) # softmax함수를 통해 확률값으로 mapping 및 최대확률값 도출
    # 레이블 인덱스 예측, max함수는 최댓값과 idx를 동시에 return
    prob, predicted_index = torch.max(probs, dim=1)  

    # 인덱스와 매칭되는 카테고리 문자열로 변환
    prob_value = prob.item()
    predicted_tag = index_to_tag[predicted_index.item()]

    return prob_value, predicted_tag

In [18]:
# 가벼운 예시로 test 결과 출력
test_input = "I hate this shit"
prob_value, predicted_tag = predict(test_input, model, word_to_index, index_to_tag)
print(prob_value, predicted_tag)

0.9992402791976929 toxicity


# Submission

In [19]:
# test file load

df_test = pd.read_csv("test_for_inference.csv")
X_test = df_test["comment"]
# 예측값을 제출형식에 맞추기 위한 list (probability와 pred 열)
prob_values = []
predicted_tags = []
for txt in tqdm(X_test):
    prob_value, predicted_tag = predict(txt, model, word_to_index, index_to_tag)
    prob_values.append(prob_value)
    predicted_tags.append(predicted_tag)
# 해당 list로 data frame 구성후 csv파일로 저장
result_df = pd.DataFrame({
    'probability': prob_values,
    'pred': predicted_tags
})
result_df.to_csv('submission.csv')

100%|███████████████████████████████████████████████████████████████████████████| 31915/31915 [00:47<00:00, 669.42it/s]
